# Remoção de senha das Notas de Corretagem

Utiliza um arquivo de senhas para descriptografar os arquivos.
O arquivo deve ter o formato:
<diretório> <senha>
    
Por exemplo:
btg_john 123
xp_jane 456
    
Quando o arquivo é descriptografado, coloca o sufixo _sem_senha e move o arquivo original para _orig.

O arquivo de diretórios deve contar um arquivo por linha.
    

In [ ]:
# para acessar lista de arquivos de um diretório
import os
from os import listdir, walk, mkdir, rename
from os.path import isfile, join

## import do PyPDF - utilizado para descriptografar os arquivos .PDF
from pikepdf import Pdf, PasswordError

## expressão regular para achar mais facilmente uma string dentro de outra
import regex

In [ ]:
ARQUIVO_SENHA = "diret_senhas.txt"
ARQUIVO_DIRETORIOS = "diretorios_notas.txt"


In [ ]:
# sufixo do arquivo descriptografado
__SUFIXO_SEM_SENHA="_sem_senha"

# diretório onde os arquivos originais (criptografados) serão movidos
__MOVE_DIR = "_orig"



In [ ]:
# processa arquivo de mapeamento de diretório e senha
__SENHAS_PDF = {}

with open(ARQUIVO_SENHA, 'r') as f:
    for linha in f:       
#        print(f"DEBUG *** linha: {linha}")
        
        if not linha.startswith("#"):
            linha_lista = linha.split()
            __SENHAS_PDF[linha_lista[0]] = linha_lista[1]


In [ ]:
# processa o arquivo de diretório de notas de corretagem a processar
diretorios_notas = []
with open(ARQUIVO_DIRETORIOS, 'r') as f:
    for linha in f:       
#        print(f"DEBUG *** linha: {linha}")
        
        if not linha.startswith("#"):
            diretorios_notas.append(linha.strip())


In [ ]:
__EXCLUDE_DIR = set([__MOVE_DIR])
diretorios = __SENHAS_PDF.keys()
re = regex.compile(r"\L<name>", name=diretorios)


In [ ]:
lista_arquivos = []
for diretorio in diretorios_notas:
    for (dirpath, dirnames, filenames) in walk(diretorio):
        dirnames[:] = [d for d in dirnames if d not in __EXCLUDE_DIR]
        filenames[:] = [f for f in filenames if __SUFIXO_SEM_SENHA not in f ]
        
            
        for f in filenames:
            saida = re.findall(dirpath)
            if (len(saida) > 0 and f.lower().endswith(".pdf")):
                lista_arquivos.append(os.path.join(dirpath,f))
            
#print(lista_arquivos)            

In [ ]:
i=1
qtd_files = len(lista_arquivos)

for arquivoPDF in lista_arquivos:
    
    print(f'Arq {i} de {qtd_files}: {arquivoPDF}')
    try:    
        arquivoCript =  Pdf.open(arquivoPDF)
        del arquivoCript
    except PasswordError as error:
        senha_pdf = __SENHAS_PDF[re.findall(arquivoPDF)[0]]
        arquivoCript =  Pdf.open(arquivoPDF, password=senha_pdf)
        
        novo_nome = arquivoPDF.split(".")[0]+__SUFIXO_SEM_SENHA+"."+arquivoPDF.split(".")[1]
        
        arquivoCript.save(novo_nome)
     
        move_diretorio = os.path.join(os.path.abspath(os.path.dirname(novo_nome)),__MOVE_DIR)
        
        if (not os.path.exists(move_diretorio)):
            mkdir(move_diretorio)
        
        old_name = os.path.basename(arquivoPDF)
    
        destino = os.path.join(move_diretorio, old_name)
        
        del arquivoCript
        
        try:
            rename(arquivoPDF, destino)
        except FileExistsError as e:
            print(f"** ERRO: arquivo {destino} já existe. O arquivo {arquivoPDF} não foi movido.")
    
    i += 1
        
print("*** Arquivos descriptografrados. ***")    

In [ ]:
for (dirpath1, dirnames1, filenames1) in walk(diretorios_notas[0]):
    print(f"dirpath1: {dirpath1}, dirnames1: {dirnames1}, filenames1:{filenames1}")
    
del dirpath1
del dirnames1
del filenames1